In [2]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

In [3]:
train_data_path = "../../data/garments_train.csv"
test_data_path = "../../data/garments_test.csv"

In [4]:
X_col, y_col = "SentimentText", "Aspect"

In [5]:
train_df = pd.read_csv(train_data_path).loc[:, [X_col, y_col]].drop_duplicates().reset_index(drop=True)
train_df.head(2)

,SentimentText,Aspect
0,사이즈가잘맞네요,사이즈
1,좀크게나온듯,사이즈


In [6]:
label_encoder = LabelEncoder()
enc_data = label_encoder.fit_transform(train_df[y_col])
num_labels = len(set(enc_data))

In [7]:
label_items = label_encoder.classes_
label_numbers = label_encoder.transform(label_items)
dict(zip(label_items, label_numbers))

{'가격': 0, '기능': 1, '디자인': 2, '사이즈': 3, '품질': 4}

In [8]:
X_train, y_train = train_df.loc[:, X_col].to_list(), enc_data

In [9]:
HUGGING_FACE_PATH = "klue/bert-base"
model = TFAutoModelForSequenceClassification.from_pretrained(HUGGING_FACE_PATH, num_labels=num_labels, from_pt=True)
tokenizer = AutoTokenizer.from_pretrained(HUGGING_FACE_PATH)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
X_train_encoding = tokenizer(X_train, padding=True, truncation=True, max_length=42)

In [11]:
SHUFFLE_PARAM = 1000

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(X_train_encoding),
    y_train
)).shuffle(SHUFFLE_PARAM)

In [12]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer, metrics=["accuracy"])
model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  110617344 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  3845      
                                                                 
Total params: 110621189 (421.99 MB)
Trainable params: 110621189 (421.99 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [13]:
BATCH_PARAM = 32

validation_length = len(X_train) // 10
train_except_val = train_dataset.skip(validation_length).batch(BATCH_PARAM)
validation_data = train_dataset.take(validation_length).batch(BATCH_PARAM)

In [14]:
model.fit(
    train_except_val,
    epochs=1,
    batch_size=BATCH_PARAM,
    validation_data=validation_data)

1267/1267 [==============================] - 439s 303ms/step - loss: 0.1858 - accuracy: 0.9401 - val_loss: 0.1288 - val_accuracy: 0.9605


In [15]:
X_col = "RawText"

In [16]:
test_df = pd.read_csv(test_data_path).loc[:, [X_col, y_col]].drop_duplicates().reset_index(drop=True)
test_df.head(2)

,RawText,Aspect
0,바늘질 마감처리 불량. 싸구려 느낌이 팍팍. 털빠짐이 없다해서 구매했는데 털빠짐이 ...,품질
1,바늘질 마감처리 불량. 싸구려 느낌이 팍팍. 털빠짐이 없다해서 구매했는데 털빠짐이 ...,기능


In [17]:
def test(x):
    labels = ["디자인", "사이즈", "가격", "품질", "기능"]
    aspects = x["Aspect"].to_list()
    result = []
    for label in labels:
        if label in aspects:
            result.append(1)
        else:
            result.append(0)
    return np.array(result)

onehot_df = test_df.groupby("RawText").apply(test).reset_index().rename(columns={ 0: "LabelList" })
onehot_df.head(2)

,RawText,LabelList
0,(12/25)생각 이상으로 아주 좋습니다. 한번 세탁해서 착용하라는 스티커대로 세탁...,"[1, 0, 0, 1, 0]"
1,*****최악최악최악최악최악***** 엄마가 선물 받아 기분 좋다고 바로 신고 나가...,"[0, 0, 0, 1, 0]"


In [20]:
X_test = onehot_df.loc[:, X_col].to_list()
X_test[:2]

['(12/25)생각 이상으로 아주 좋습니다. 한번 세탁해서 착용하라는 스티커대로 세탁해서 입죠. 디자인 좋고 니트 짜임새 느낌 좋네요. 굿 입니다. (12/28) 세탁 했는데도 많이 묻어납니다.  두번째로 세탁하고 있습니다. 건조 후 확인하고 내용 업데이트하죠 (12/29) 2번 세탁해도 기모털이 많이 묻어나서 못 입겠다는 결론... 반품 바랍니다',
 '*****최악최악최악최악최악***** 엄마가 선물 받아 기분 좋다고 바로 신고 나가심 외부착화는 당연히 잘못된거 알지만 좌우가 눈에 보이게 한쪽이 헐떡헐떡 벗겨져서 고객센터 연락했더니 자로재서 숫자 알려달라함 알려줬더니 사진 찍어 보내라함 이번엔 자로잰 사진으로는 확인이 안된다며 두짝을 겹쳐 찍으라 세번째 요청이 옴 원래 짝발도 아니고 한쪽이 그렇게 헐떡거리면 분명 제품의 품질 문제라고 생각됨 직통번호도 없어서 두번이나 연락처 남기고 전화 기다렸는데 안오고 내가 말하는데 상담원이 이중으로 말해서 더 기분이 안좋아짐 선물하고 기분더러움']

In [21]:
X_test_encoding = tokenizer(X_test, padding=True, truncation=True, max_length=42)

In [22]:
test_dataset = tf.data.Dataset.from_tensor_slices(
    dict(X_test_encoding)
).batch(BATCH_PARAM)

In [23]:
predictions = model.predict(test_dataset)
predictions.logits

222/222 [==============================] - 26s 102ms/step


array([[-1.4607196 , -1.7143906 ,  4.7414055 , -2.557926  ,  1.0661823 ],
       [-2.0755408 , -2.4306877 ,  1.2957723 , -1.8196414 ,  4.6529875 ],
       [-2.6485467 , -2.7624114 ,  1.0066824 ,  5.8418493 , -0.7810662 ],
       ...,
       [-3.0679405 , -1.1401069 ,  2.9347847 ,  0.6007072 ,  0.29108593],
       [-2.7304518 , -2.7799816 ,  1.0598868 ,  5.985725  , -0.83527696],
       [-3.9425514 , -0.73929393,  2.0418615 ,  3.121753  , -0.738873  ]],
      dtype=float32)

In [30]:
def logits_to_onehot_aspects(logits, threshold):
    sig_probabilities = tf.keras.activations.sigmoid(logits).numpy()
    return list(np.where(sig_probabilities > threshold, 1, 0))

aspect_bools = logits_to_onehot_aspects(predictions.logits, 0.6)
pred_series = pd.Series(aspect_bools)
pred_series.head()

0    [0, 0, 1, 0, 1]
1    [0, 0, 1, 0, 1]
2    [0, 0, 1, 1, 0]
3    [0, 0, 1, 1, 0]
4    [0, 0, 1, 1, 0]
dtype: object

In [31]:
onehot_df["PredList"] = pred_series
onehot_df.head(2)

,RawText,LabelList,PredList
0,(12/25)생각 이상으로 아주 좋습니다. 한번 세탁해서 착용하라는 스티커대로 세탁...,"[1, 0, 0, 1, 0]","[0, 0, 1, 0, 1]"
1,*****최악최악최악최악최악***** 엄마가 선물 받아 기분 좋다고 바로 신고 나가...,"[0, 0, 0, 1, 0]","[0, 0, 1, 0, 1]"


In [32]:
test_series = onehot_df.apply(lambda x: x["LabelList"] == x["PredList"], axis=1)
test_series.head()

0    [False, True, False, False, False]
1     [True, True, False, False, False]
2     [True, False, False, False, True]
3     [True, False, False, False, True]
4       [True, True, True, False, True]
dtype: object

In [33]:
# 완전일치
def check_full_accord(x):
    for each in x:
        if not each:
            return 0
    return 1

test_series.apply(check_full_accord).mean()

0.0400113106178425

In [34]:
# 부분일치
def check_partial_accord(x):
    result = 0
    for each in x:
        if each:
            result += 1
    return result / 5

test_series.apply(check_partial_accord).mean()

0.48522550544323484